In [1]:
import pandas as pd
import numpy as np
import util
import preprocessing
import importlib
import string
import re
import nlp_util
##java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

NLP_Task ready to use.


In [2]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
liwc_data = pd.read_csv('..//dataset//train_liwc.csv')

In [27]:
liwc_data_va = pd.read_csv('..//dataset//valid_liwc.csv')

In [36]:
liwc_data

,A,B,C,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,0,1,2,3,4.00,5.00,6.00,7.00,8.00,9.00,...,86.00,87.00,88.0,89.00,90.0,91.00,92.00,93.00,94.00,95.00
1,2635.json,FALSE,Says the Annies List political group supports ...,12,99.00,95.20,1.79,99.00,12.00,33.33,...,0.00,0.00,0.0,0.00,0.0,8.33,0.00,0.00,0.00,0.00
2,10540.json,half-true,When did the decline of coal start? It started...,24,80.75,50.00,98.01,25.77,8.00,25.00,...,0.00,0.00,0.0,4.17,0.0,0.00,0.00,0.00,8.33,0.00
3,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",19,99.00,70.08,1.00,97.58,19.00,15.79,...,0.00,0.00,0.0,0.00,0.0,0.00,10.53,0.00,0.00,0.00
4,1123.json,FALSE,Health care reform legislation is likely to ma...,12,92.84,50.00,1.79,99.00,12.00,25.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
5,9028.json,half-true,The economic turnaround started at the end of ...,10,99.00,15.86,99.00,25.77,10.00,30.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
6,12465.json,TRUE,The Chicago Bears have had more starting quart...,27,99.00,35.54,92.47,1.00,27.00,22.22,...,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,7.41,0.00
7,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,13,82.00,22.08,94.81,25.77,13.00,15.38,...,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
8,153.json,half-true,I'm the only person on this stage who has work...,27,92.84,64.46,58.07,88.52,27.00,18.52,...,7.41,0.00,0.0,0.00,0.0,0.00,0.00,3.70,0.00,0.00
9,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",24,98.02,33.83,74.76,25.77,24.00,29.17,...,4.17,0.00,0.0,0.00,0.0,4.17,0.00,0.00,0.00,4.17


In [20]:
liwc_data.iloc[:,5:]

,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,5.00,6.00,7.00,8.00,9.00,10.00,11.00,12.00,13.00,14.00,...,86.00,87.00,88.0,89.00,90.0,91.00,92.00,93.00,94.00,95.00
1,95.20,1.79,99.00,12.00,33.33,75.00,16.67,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,8.33,0.00,0.00,0.00,0.00
2,50.00,98.01,25.77,8.00,25.00,75.00,41.67,8.33,0.00,0.00,...,0.00,0.00,0.0,4.17,0.0,0.00,0.00,0.00,8.33,0.00
3,70.08,1.00,97.58,19.00,15.79,63.16,36.84,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,10.53,0.00,0.00,0.00
4,50.00,1.79,99.00,12.00,25.00,75.00,16.67,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
5,15.86,99.00,25.77,10.00,30.00,90.00,50.00,10.00,10.00,10.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
6,35.54,92.47,1.00,27.00,22.22,81.48,37.04,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,7.41,0.00
7,22.08,94.81,25.77,13.00,15.38,76.92,53.85,7.69,7.69,0.00,...,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00
8,64.46,58.07,88.52,27.00,18.52,77.78,48.15,11.11,3.70,3.70,...,7.41,0.00,0.0,0.00,0.0,0.00,0.00,3.70,0.00,0.00
9,33.83,74.76,25.77,24.00,29.17,70.83,37.50,4.17,0.00,0.00,...,4.17,0.00,0.0,0.00,0.0,4.17,0.00,0.00,0.00,4.17


In [101]:
liwc_features = np.zeros((len(liwc_data),len(liwc_data.columns)))
#for i in range(3,len(liwc_data.columns)):
for i in range(len(liwc_data.columns)):    
    #print(liwc_data.iloc[:,i].values)
    #print(liwc_features[:,i-3])
    if i not in [0,1,2]:#,27,28,30,23,31,6,10,67]:
        liwc_features[:,i]=liwc_data.iloc[:,i].values

In [102]:
liwc_features_va = np.zeros((len(liwc_data_va),len(liwc_data_va.columns)))
#for i in range(3,len(liwc_data.columns)):
for i in range(len(liwc_data_va.columns)):    
    #print(liwc_data.iloc[:,i].values)
    #print(liwc_features[:,i-3])
    if i not in [0,1]:#,27,28,30,23,31,6,10,67]:
        liwc_features_va[:,i]=liwc_data_va.iloc[:,i].values

In [55]:
# Multiclassifier Labels
#multi_labels_tr = np.array(preprocessing.create_labels(labels=labels,label_values={'false':1, 'true':2,'pants-fire':3,'barely-true':4,'half-true':5,'mostly-true':6}))
#multi_labels_va = np.array(preprocessing.create_labels(labels=va_dict['label'],label_values={'false':1, 'true':2,'pants-fire':3,'barely-true':4,'half-true':5,'mostly-true':6}))

# Binary Labels
binary_labels_tr = np.array(preprocessing.create_labels(labels=liwc_data.iloc[1:,1].values,label_values={'FALSE':1, 'TRUE':-1,'pants-fire':1,'barely-true':1,'half-true':0,'mostly-true':-1}))
binary_labels_va = np.array(preprocessing.create_labels(labels=liwc_data_va.iloc[1:,1].values,label_values={'FALSE':1, 'TRUE':-1,'pants-fire':1,'barely-true':1,'half-true':0,'mostly-true':-1}))

In [54]:
liwc_data_va.iloc[1:,1].values

array(['pants-fire', 'FALSE', 'half-true', ..., 'TRUE', 'FALSE',
       'barely-true'], dtype=object)

In [9]:
liwc_data.iloc[1:,4].values[(liwc_data.iloc[1:,1].values=='FALSE')]

array([99.  , 92.84, 20.93, ..., 24.86, 99.  , 99.  ])

In [10]:
liwc_data.iloc[1:,4].values[(liwc_data.iloc[1:,1].values=='TRUE')]

array([99.  , 78.35, 28.22, ..., 98.92, 92.84, 99.  ])

In [11]:
liwc_data.iloc[1:,4].values[(liwc_data.iloc[1:,1].values=='half-true')]

array([80.75, 99.  , 92.84, ..., 99.  , 89.9 , 98.65])

In [12]:
liwc_data.iloc[1:,4].values[(liwc_data.iloc[1:,1].values=='pants-fire')]

array([99.  , 99.  , 99.  , 17.96, 67.14, 99.  , 99.  , 74.89, 99.  ,
       97.54, 92.84,  9.98, 82.52, 99.  , 99.  , 99.  , 99.  , 54.96,
       92.84, 84.57, 92.84, 99.  , 92.84, 92.84, 92.84, 89.9 , 82.  ,
       84.23, 15.89, 99.  , 99.  , 92.84, 99.  , 76.19, 77.33, 82.  ,
       17.96, 97.44, 99.  , 97.89, 99.  , 99.  , 92.84,  2.91, 60.8 ,
       99.  , 98.92, 84.57, 99.  , 96.54, 98.6 , 92.84, 92.84, 99.  ,
       99.  , 98.92, 99.  , 99.  , 70.3 , 99.  , 79.25, 98.46, 92.84,
       99.  , 92.84, 37.39, 97.77, 92.84, 99.  , 97.44, 99.  , 92.84,
       97.77, 47.35, 24.18, 96.69, 99.  , 98.6 , 70.3 , 98.98, 98.92,
       92.84, 92.84, 81.4 , 92.84, 58.07, 92.84, 99.  , 98.75, 83.44,
       28.22, 99.  , 97.77, 99.  , 92.84, 99.  , 65.78, 40.12, 97.89,
       92.84, 92.84, 92.84, 82.  , 97.54, 92.84, 99.  , 79.25, 99.  ,
        1.  , 99.  , 98.46, 99.  , 99.  , 92.84, 92.84, 97.54, 64.27,
       99.  , 23.89, 12.85, 97.54, 28.22, 74.89, 71.61, 83.85, 98.02,
       92.84, 98.75,

In [13]:
liwc_data.iloc[1:,4].values[(liwc_data.iloc[1:,1].values=='mostly-true')]

array([99.  , 92.84, 99.  , ..., 98.75, 97.02, 92.84])

In [14]:
liwc_data.iloc[1:,4].values[(liwc_data.iloc[1:,1].values=='barely-true')]

array([82.  , 71.61, 70.3 , ..., 98.75, 99.  , 99.  ])

In [103]:
Xtrain = liwc_features[1:,3:][(binary_labels_tr!=0)]
Xdev = liwc_features_va[1:,2:][(binary_labels_va!=0)]
Ytrain = binary_labels_tr[(binary_labels_tr!=0)]
Ydev = binary_labels_va[(binary_labels_va!=0)]

In [109]:
# Feature values
feature_values = liwc_data.columns[3:]

In [76]:
Ytrain

array([ 1, -1,  1, ..., -1,  1,  1])

In [81]:
from sklearn.dummy import DummyClassifier

In [83]:
dummyClassifier = DummyClassifier(strategy='most_frequent')
dummyClassifier.fit(Xtrain,Ytrain)
pred= dummyClassifier.predict(Xdev)
print('Accuracy: ', np.mean(pred==Ydev))

Accuracy:  0.5942028985507246


In [77]:
from sklearn.tree import DecisionTreeClassifier

In [189]:
Xtrain_pruned = Xtrain_pruned[:,pruned_features]
Xdev_pruned = Xdev_pruned[:,pruned_features]
features_names_pruned = features_names_pruned[pruned_features]

In [194]:
#for i in range(1,10):
#print('Depth: ', i)
clf = DecisionTreeClassifier(max_depth = 3)
clf.fit(Xtrain_pruned[:,pruned_features],Ytrain)
pred= clf.predict(Xdev_pruned[:,pruned_features])
print('Accuracy: ', np.mean(Ydev==pred))
print(classification_report(Ydev, pred))


Accuracy:  0.5826086956521739
             precision    recall  f1-score   support

         -1       0.44      0.10      0.16       420
          1       0.60      0.91      0.72       615

avg / total       0.53      0.58      0.49      1035



In [190]:
#clf.feature_importances_[(clf.feature_importances_<=0.1)]
pruned_features = (clf.feature_importances_<=0.1)

In [195]:
for i,x in enumerate(clf.feature_importances_):
    if x>=0.1:
        print(i, x, features_names_pruned[pruned_features][i])

1 0.15553516562339995 Clout
3 0.10679638500918288 pronoun
11 0.10152122353053993 article
17 0.15326760317186 affect
27 0.125277559183767 insight
45 0.10669456057547042 focuspast
48 0.2509075029057799 space


### Getting LIWC features

After 1st Iteration <br>
Threshold: 0.1 <br>
25 0.23726303653426356 compare<br>
27 0.33371462903160654 number<br>
28 0.10161172852929526 quant<br>
64 0.10310744904200508 focusfuture<br>

2nd Iteration<br>
24 0.26498982893332584 adj<br>
43 0.11275220316117086 differ<br>
64 0.1309803833643041 time<br>

3rd Iteration<br>
3 0.5387284840339214 Authentic<br>
5 0.18278713929091212 WPS<br>
20 0.27848437667516657 adverb<br>

4th Iteration<br>
0 0.20852693514504003 WC<br>
39 0.11920900043889934 percept<br>
56 0.20756330954331384 relativ<br>

5th: remove 4 1.0 Dic<br>

6th Iteration <br>
3 0.1144315107744801 Sixltr<br>
21 0.14490825323456402 posemo<br>
38 0.1732489421085719 hear<br>
59 0.11866405259630831 relig<br>

7th Iteration <br>
3 0.1699224428235141 function<br>
17 0.14409875722257276 verb<br>
24 0.11210971402129889 social<br>

8th Iteration <br>
1 0.15553516562339995 Clout<br>
3 0.10679638500918288 pronoun<br>
11 0.10152122353053993 article<br>
17 0.15326760317186 affect<br>
27 0.125277559183767 insight<br>
45 0.10669456057547042 focuspast<br>
48 0.2509075029057799 space<br>


In [50]:
liwc_features_va[1:,2:]

array([[26.  , 23.17, 22.08, ...,  0.  ,  7.69,  0.  ],
       [32.  , 63.62, 94.08, ...,  0.  ,  0.  ,  0.  ],
       [ 8.  , 71.61, 99.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [13.  , 42.69, 22.08, ...,  0.  ,  0.  ,  0.  ],
       [32.  , 95.4 , 96.95, ...,  3.12,  0.  ,  0.  ],
       [17.  , 97.02, 27.83, ...,  0.  ,  0.  ,  0.  ]])